In [ ]:
import cv2
import pytesseract
import os
import csv

# Chemin vers le dossier contenant les photos
dossier_photos = r"/Users/dimitriraymond/Documents/Epitech/Semestre 2/DAT/image_test"

In [1]:
# Liste pour stocker les résultats
resultats = []

# Parcourir les fichiers du dossier de photos
for fichier in os.listdir(dossier_photos):
    # Vérifier si le fichier est une image
    if fichier.lower().endswith((".jpg", ".jpeg", ".png")):
        # Chemin complet vers l'image
        chemin_image = os.path.join(dossier_photos, fichier)

        # Chargement de l'image
        img = cv2.imread(chemin_image)

        # Conversion de l'image en niveaux de gris
        gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Application d'un seuillage pour obtenir un masque binaire
        _, binary_img = cv2.threshold(gray_img, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

        # Suppression du bruit avec une opération de fermeture morphologique
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
        closed_img = cv2.morphologyEx(binary_img, cv2.MORPH_CLOSE, kernel)

        # Recherche des contours dans l'image
        contours, _ = cv2.findContours(closed_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Parcours des contours et extraction des titres
        titles = []
        for contour in contours:
            x, y, w, h = cv2.boundingRect(contour)
            if w > 100 and h > 20:  # Filtre pour exclure les petits contours indésirables
                title_img = gray_img[y:y+h, x:x+w]
                title_text = pytesseract.image_to_string(title_img)
                titles.append(title_text.strip())

        # Analyse de la présence de chiffre dans le titre
        has_number = any(any(char.isdigit() for char in title) for title in titles)

        # Ajouter le résultat dans la liste des résultats
        resultats.append({"image": fichier, "number": int(has_number)})

# Écrire les résultats dans un fichier CSV
chemin_fichier_csv = r"/Users/dimitriraymond/Documents/Epitech/Semestre 2/DAT/resultc/data483.csv"
with open(chemin_fichier_csv, mode='w', newline='') as fichier_csv:
    writer = csv.DictWriter(fichier_csv, fieldnames=["image", "number"])
    writer.writeheader()  # Écrire l'en-tête du fichier CSV
    writer.writerows(resultats)  # Écrire les résultats

print("Terminé. Les résultats ont été enregistrés dans", chemin_fichier_csv)

Terminé. Les résultats ont été enregistrés dans /Users/dimitriraymond/Documents/Epitech/Semestre 2/DAT/resultc/data483.csv
